In [1]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [ ]:
%mkdir ch20
!touch ch20/__init__.py

## 20.0 序言

- 描述符是对多个属性运用相同存取逻辑的一种方式
- 描述符是实现了特定协议的类
  - 这个协议包括 `__get__`、`__set__` 和`__delete__` 方法
- 描述符是 `Python` 的独有特征,不仅在应用层中使用,在语言的基础设施中也有用到
  - 除了特性之外,使用描述符的 Python 功能还有方法及`classmethod` 和 `staticmethod` 装饰器



## 20.1 描述符示例：验证属性

- 特性工厂函数借助函数式编程模式避免重复编写读值方法和设值方法
  - 特性工厂函数是高阶函数
- 解决这种问题的面向对象方式是描述符类

### 20.1.1 `LineItem` 类第 3版: 一个简单的描述符

- 描述符的用法是,创建一个实例, 作为另一个类的类属性
- UML 图
  - ![UML 图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200810115546.png)
- 相关定义
  - 描述符类
    - 实现描述符协议的类
  - 托管类
    - 把描述符实例声明为类属性的类
  - 描述符实例
    - 描述符类的各个实例,声明为托管类的类属性
    - 在 UML 中
      - 下划线表示类属性
      - 与黑色菱形接触的 `LineItem` 类包含描述符实例
  - 托管实例
    - 托管类的实例
  - 储存属性
    - 托管实例中存储自身托管属性的属性
  - 托管属性
    - 托管类中由描述符实例处理的公开属性,值存储在储存属性中
    - 描述符实例和储存属性为托管属性建立了基础
    

- Quantity 实例是 LineItem 类的类属性
  - ![示意图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200810120132.png)
  - ![托管属性](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200810120437.png)

###### 示例 20-1 bulkfood_v3.py:使用 `Quantity` 描述符管理 `LineItem`

In [ ]:
%%writefile ch20/bulkfood_v3.py
class Quantity:  # ➊ 描述符基于协议实现,无需创建子类
  def __init__(self, storage_name):
    self.storage_name = storage_name  # ➋ storage_name 属性是托管实例中存储值的属性的名称

  def __set__(self, instance, value):  # ➌ 尝试为托管属性赋值时,会调用 __set__ 方法; self 是描述符实例, 也是托管类的类属性; instance 是托管实例(LineItem 实例),value 是要设定的值
    if value > 0:
      instance.__dict__[self.storage_name] = value  # ➍ 必须直接处理托管实例的 __dict__ 属性;如果使用内置的 setattr 函数,会再次触发 __set__ 方法,导致无限递归. 因为托管属性和实际属性的名称相同
    else:
      raise ValueError('value must be > 0') 


class LineItem:
  weight = Quantity('weight')  # ➎ 第一个描述符实例绑定给 weight 属性
  price = Quantity('price')  # ➏ 第二个描述符实例绑定给 price 属性

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Overwriting ch20/bulkfood_v3.py


In [ ]:
from ch20.bulkfood_v3 import LineItem

In [ ]:
truffle = LineItem('White truffle', 100, 0)

ValueError: ignored

### 20.1.2 `LineItem` 类第4版:自动获取储存属性的名称

- ![UML 类图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200812104356.png)
- 为了避免在描述符声明语句中重复输入属性名, 为每个`Quantity` 实例的 `storage_name` 属性生成一个独一无二的字符串
- 以 `_Quantity#` 为前缀, 然后在后面拼接一个整数
  - 在前缀中使用井号能避免 `storage_name` 与用户使用点号创建的属性冲突
  - `#` 相当于是注释


###### 示例 20-2 bulkfood_v4.py:每个 `Quantity` 描述符都有独一无二的 `storage_name`

- 内置的高阶函数可以直接使用 `getattr` 和 `setattr` 存取值, 无需使用`instance.__dict__`
  - 因为托管属性和储存属性的名称不同,  所以把储存属性传给 `getattr` 函数不会触发描述符


In [ ]:
%%writefile ch20/bulkfood_v4.py
class Quantity:
  __counter = 0  # ➊ __counter 是 Quantity 类的类属性,统计 Quantity 实例的数量
  def __init__(self):
    cls = self.__class__  # ➋ cls 是 Quantity 类的引用
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)  # ➌ 每个描述符实例的 storage_name 属性都是独一无二的,因为其值由描述符类的名称和 __counter 属性的当前值构成(例如,_Quantity#0) 
    cls.__counter += 1  # ➍ 递增 __counter 属性的值

  def __get__(self, instance, owner):  # ➎ 需要要实现 __get__ 方法,因为托管属性的名称与 storage_name 不同
    return getattr(instance, self.storage_name)  # ➏ 使用内置的 getattr 函数从 instance 中获取储存属性的值

  def __set__(self, instance, value):
    if value > 0:
      setattr(instance, self.storage_name, value)  # ➐ 使用内置的 setattr 函数把值存储在 instance 中
    else:
      raise ValueError('value must be > 0')


class LineItem:
  weight = Quantity()  # ➑ 不用再把托管属性的名称传给 Quantity 构造方法
  price = Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Writing ch20/bulkfood_v4.py


In [ ]:
from ch20.bulkfood_v4 import LineItem

In [ ]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)
coconuts.weight, coconuts.price

(20, 17.95)

In [ ]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

(20, 17.95)

###### 示例 20-3 bulkfood_v4b.py(只列出部分代码):通过托管类调用时, `__get__` 方法返回描述符的引用

- `__get__` 方法有三个参数:`self`、`instance` 和 `owner`
  - `owner`参数是托管类(如 `LineItem)`的引用, 通过描述符从托管类中获取属性时用得到
  - 如果使用 `LineItem.weight` 从类中获取托管属性(以`weight` 为例),描述符的 `__get__` 方法接收到的 `instance` 参数值是`None`
  - 因此,下述控制台会话会抛出 `AttributeError` 异常


In [ ]:
from ch20.bulkfood_v4 import LineItem
LineItem.weight

AttributeError: ignored

- 为了给用户提供内省和其他元编程技术支持,通过类访问托管属性时,最好让 `__get__` 方法返回描述符实例

In [ ]:
%%writefile ch20/bulkfood_v4b.py

class Quantity:
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self  # ➊ 如果不是通过实例调用,返回描述符自身
    else:
      return getattr(instance, self.storage_name)  # ➋ 否则,像之前一样,返回托管属性的值

  def __set__(self, instance, value):
    if value > 0:
      setattr(instance, self.storage_name, value)
    else:
      raise ValueError('value must be > 0')


class LineItem:
  weight = Quantity()
  price = Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Overwriting ch20/bulkfood_v4b.py


In [ ]:
from ch20.bulkfood_v4b import LineItem
LineItem.price

In [ ]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

###### 示例 20-4 bulkfood_v4c.py:整洁的 `LineItem` 类; `Quantity` 描述符类现在位于导入的 `model_v4c` 模块中

- 类似于模型定义， Django 模型的字段就是描述符

In [ ]:
%%writefile ch20/model_v4c.py
class Quantity:
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self  # ➊ 如果不是通过实例调用,返回描述符自身
    else:
      return getattr(instance, self.storage_name)  # ➋ 否则,像之前一样,返回托管属性的值

  def __set__(self, instance, value):
    if value > 0:
      setattr(instance, self.storage_name, value)
    else:
      raise ValueError('value must be > 0')

Writing ch20/model_v4c.py


In [2]:
%%writefile ch20/bulkfood_v4c.py
import ch20.model_v4c as model


class LineItem:
  weight = model.Quantity()
  price = model.Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Overwriting ch20/bulkfood_v4c.py


In [ ]:
from ch20.bulkfood_v4c import LineItem
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

##### 特性工厂函数与描述符类比较

- 从某种程度上来讲, 特性工厂函数模式较简单, 可是描述符类方式更易扩展, 而且应用也更广泛

###### 示例 20-5 bulkfood_v4prop.py:使用特性工厂函数实现与示例 20-2 中的描述符类相同的功能

In [ ]:
%%writefile ch20/bulkfood_v4prop.py
def quantity():  # ➊ 没有 storage_name 参数
  try:
    quantity.counter += 1  # ➋ 为了在多次调用之间共享 counter,因此把它定义为 quantity 函数自身的属性
  except AttributeError:
    quantity.counter = 0  # ➌ 如果 quantity.counter 属性未定义,把值设为 0
  storage_name = '_{}:{}'.format('quantity', quantity.counter)  # ➍ 创建一个局部变量 storage_name,借助闭包保持它的值,供后面的 qty_getter 和 qty_setter 函数使用

  def qty_getter(instance):  # ➎ 使用内置的 getattr 和 setattr 函数实现相关功能
    return getattr(instance, storage_name)

  def qty_setter(instance, value):
    if value > 0:
      setattr(instance, storage_name, value)
    else:
      raise ValueError('value must be > 0')
  
  return property(qty_getter, qty_setter)


class LineItem:
  weight = quantity()
  price = quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.pric

Overwriting ch20/bulkfood_v4prop.py


In [ ]:
from ch20.bulkfood_v4prop import LineItem
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

### 20.1.3 `LineItem` 类第5版:一种新型描述符

- ![UML 类图](https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200812105602.png)
- `Validated`、`Quantity` 和 `NonBlank` 三个类之间的关系体现了模板方法设计模式
> 一个模板方法用一些抽象的操作定义一个算法, 而子类将重定义这些操作以提供具体的行为
- `LineItem` 示例演示了描述符的典型用途——管理数据属性


###### 示例 20-6 model_v5.py: 重构后的描述符类

In [ ]:
%%writefile ch20/model_v5.py
import abc


class AutoStorage:  # ➊ AutoStorage 类提供了之前 Quantity 描述符的大部分功能
  __counter = 0

  def __init__(self):
    cls = self.__class__
    prefix = cls.__name__
    index = cls.__counter
    self.storage_name = '_{}#{}'.format(prefix, index)
    cls.__counter += 1

  def __get__(self, instance, owner):
    if instance is None:
      return self
    else:
      return getattr(instance, self.storage_name)

  def __set__(self, instance, value):
      setattr(instance, self.storage_name, value)  # ➋ ......验证除外


class Validated(abc.ABC, AutoStorage):  # ➌ Validated 是抽象类,不过也继承自 AutoStorage 类
  def __set__(self, instance, value):
    value = self.validate(instance, value)  # ➍ __set__ 方法把验证操作委托给 validate 方法......
    super().__set__(instance, value)  # ➎ ......然后把返回的 value 传给超类的 __set__ 方法,存储值
    
  @abc.abstractmethod
  def validate(self, instance, value):  # ➏ validate 是类的抽象方法
    """return validated value or raise ValueError"""

  
class Quantity(Validated):  # ➐ Quantity 和 NonBlank 都继承自 Validated 类
  """a number greater than zero"""
  def validate(self, instance, value):
    if value <= 0:
      raise ValueError('value must be > 0')
    return value


class NonBlank(Validated):
  """a string with at least one non-space character"""
  def validate(self, instance, value):
    value = value.strip()
    if len(value) == 0:
      raise ValueError('value cannot be empty or blank')
    return value  # ➑ 要求具体的 validate 方法返回验证后的值,借机可以清理、转换或规范化接收的数据。这里把 value 首尾的空白去掉,然后将其返回

Writing ch20/model_v5.py


###### 示例 20-7 bulkfood_v5.py:使用 Quantity 和 NonBlank 描述符的 LineItem 类

In [ ]:
%%writefile ch20/bulkfood_v5.py
import ch20.model_v5 as model  # ➊ 导入 model_v5 模块,指定一个更友好的名称


class LineItem:
  description = model.NonBlank()  # ➋ 使用 model.NonBlank 描述符。其余的代码没变
  weight = model.Quantity()
  price = model.Quantity()

  def __init__(self, description, weight, price):
    self.description = description
    self.weight = weight
    self.price = price

  def subtotal(self):
    return self.weight * self.price

Overwriting ch20/bulkfood_v5.py


In [ ]:
from ch20.bulkfood_v5 import LineItem
br_nuts = LineItem('  ', 10, 34.95)

ValueError: ignored

In [ ]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

## 20.2 覆盖型与非覆盖型描述符对比

- Python 存取属性的方式是不对等
  - 通过实例读取属性时, 通常返回的是实例中定义的属性; 但是, 如果实例中没有指定的属性, 那么会获取类属性
  - 而为实例中的属性赋值时, 通常会在实例中创建属性, 根本不影响类
- 这种不对等的处理方式对描述符也有影响。根据是否定义`__set__` 方法, 描述符可分为两大类
  - 覆盖型描述符
  - 非覆盖型描述符



###### 示例 20-8 descriptorkinds.py:几个简单的类,用于研究描述符的覆盖行为

In [ ]:
%%writefile ch20/descriptorkinds.py
### 辅助函数,仅用于显示 ###


def cls_name(obj_or_cls):
  cls = type(obj_or_cls)
  if cls is type:
    cls = obj_or_cls
  return cls.__name__.split('.')[-1]


def display(obj):
  cls = type(obj)
  if cls is type:
    return '<class {}>'.format(obj.__name__)
  elif cls in [type(None), int]:
    return repr(obj)
  else:
    return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
  pseudo_args = ', '.join(display(x) for x in args)
  print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))


### 对这个示例重要的类 ###


class Overriding:  # ➊ 有 __get__ 和 __set__ 方法的典型覆盖型描述符
  """也称数据描述符或强制描述符"""

  def __get__(self, instance, owner):
    print_args('get', self, instance, owner)  # ➋ 在这个示例中,各个描述符的每个方法都调用了 print_args 函数
  def __set__(self, instance, value):
    print_args('set', self, instance, value)


class OverridingNoGet:  # ➌ 没有 __get__ 方法的覆盖型描述符
  """没有``__get__``方法的覆盖型描述符"""
  def __set__(self, instance, value):
    print_args('set', self, instance, value)


class NonOverriding:  # ➍ 没有 __set__ 方法,所以这是非覆盖型描述符
  """也称非数据描述符或遮盖型描述符"""
  def __get__(self, instance, owner):
    print_args('get', self, instance, owner)


class Managed:  # ➎ 托管类,使用各个描述符类的一个实例
  over = Overriding()
  over_no_get = OverridingNoGet()
  non_over = NonOverriding()

  def spam(self):  # ➏ spam 方法放在这里是为了对比,因为方法也是描述符
    print('-> Managed.spam({})'.format(display(self)))

Overwriting ch20/descriptorkinds.py


### 20.2.1 覆盖型描述符

- 实现 `__set__` 方法的描述符属于覆盖型描述符
- 虽然描述符是类属性, 但是实现 `__set__` 方法的话, 会覆盖对实例属性的赋值操作


###### 示例 20-9 覆盖型描述符的行为,其中 obj.over 是 Overriding 类(见示例 20-8)的实例

In [ ]:
from ch20.descriptorkinds import Managed

In [ ]:
obj = Managed()
obj.over  # obj.over 触发描述符的 __get__ 方法,第二个参数的值是托管实例 obj

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [ ]:
Managed.over  # Managed.over 触发描述符的 __get__ 方法,第二个参数(instance)的值是 None

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [ ]:
obj.over = 7  # 为 obj.over 赋值,触发描述符的 __set__ 方法,最后一个参数的值是 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [ ]:
obj.over  # 读取 obj.over,仍会触发描述符的 __get__ 方法

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [ ]:
obj.__dict__['over'] = 8  # 跳过描述符,直接通过 obj.__dict__ 属性设值
vars(obj)  # 确认值在 obj.__dict__ 属性中,在 over 键名下

{'over': 8}

In [ ]:
obj.over  # 然而,即使是名为 over 的实例属性,Managed.over 描述符仍会覆盖读取 obj.over 这个操作

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


### 20.2.2 没有 `__get__` 方法的覆盖型描述符

- 实例属性会遮盖描述符,不过只有读操作是如此
  - 如果直接通过实例的 `__dict__`属性创建同名实例属性, 以后再设置那个属性时, 仍会由 `__set__` 方法插手接管
  - 但是读取那个属性的话, 就会直接从实例中返回新赋予的值, 而不会返回描述符对象


###### 示例 20-10 没有 `__get__` 方法的覆盖型描述符

In [ ]:
obj.over_no_get  # 这个覆盖型描述符没有 __get__ 方法,因此,obj.over_no_get 从类中获取描述符实例

In [ ]:
Managed.over_no_get  # 直接从托管类中读取描述符实例也是如此

In [ ]:
obj.over_no_get = 7  # 为 obj.over_no_get 赋值会触发描述符的 __set__ 方法

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [ ]:
obj.over_no_get  # 因为 __set__ 方法没有修改属性,所以在此读取 obj.over_no_get获取的仍是托管类中的描述符实例

In [ ]:
obj.__dict__['over_no_get'] = 9  # 通过实例的 __dict__ 属性设置名为 over_no_get 的实例属性
obj.over_no_get  # 现在,over_no_get 实例属性会遮盖描述符,但是只有读操作是如此

9

In [ ]:
obj.over_no_get = 7  # 为 obj.over_no_get 赋值,仍然经过描述符的 __set__ 方法处理

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [ ]:
obj.over_no_get  # 但是读取时,只要有同名的实例属性,描述符就会被遮盖

9

### 20.2.3 非覆盖型描述符

- 没有实现 `__set__` 方法的描述符是非覆盖型描述符
  - 如果设置了同名的实例属性, 描述符会被遮盖, 致使描述符无法处理那个实例的那个属性


###### 示例 20-11 非覆盖型描述符的行为

In [ ]:
obj.non_over  # obj.non_over 触发描述符的 __get__ 方法,第二个参数的值是 obj

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [ ]:
obj.non_over = 7  # Managed.non_over 是非覆盖型描述符,因此没有干涉赋值操作的__set__ 方法
obj.non_over  # 现在,obj 有个名为 non_over 的实例属性,把 Managed 类的同名描述符属性遮盖掉

7

In [ ]:
Managed.non_over  # Managed.non_over 描述符依然存在,会通过类截获这次访问

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [ ]:
del obj.non_over  # 删除 non_over 实例属性
obj.non_over  # 读取 obj.non_over 时,会触发类中描述符的 __get__ 方法

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


### 20.2.4 在类中覆盖描述符

- 不管描述符是不是覆盖型,为类属性赋值都能覆盖描述符
  - 这是一种猴子补丁技术
- 类读写属性也是不对等
  - 读类属性的操作可以由依附在托管类上定义有 `__get__` 方法的描述符处理
  - 但是写类属性的操作不会由依附在托管类上定义有 `__set__` 方法的描述符处理


###### 示例 20-12 通过类可以覆盖任何描述符

In [ ]:
obj = Managed()
Managed.over = 1  # 覆盖类中的描述符属性
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over  # 描述符不见了

(1, 2, 3)

## 20.3 方法是描述符

- 在类中定义的函数属于绑定方法`(bound method)`
  - 因为用户定义的函数都有 `__get__` 方法, 所以依附到类上时, 就相当于非覆盖型描述符
- 函数会变成绑定方法, 这是 Python 语言底层使用描述符的最好例证


###### 示例 20-13 方法是非覆盖型描述符

In [ ]:
obj = Managed()
obj.spam  # obj.spam 获取的是绑定方法对象

<bound method Managed.spam of <ch20.descriptorkinds.Managed object at 0x7fbc838a0a90>>

In [ ]:
Managed.spam  # Managed.spam 获取的是函数

<function ch20.descriptorkinds.Managed.spam>

In [ ]:
obj.spam = 7  # 如果为 obj.spam 赋值,会遮盖类属性,导致无法通过 obj 实例访问 spam 方法

In [ ]:
obj.spam

7

###### 示例 20-14 method_is_descriptor.py:Text 类,继承自 UserString 类

In [ ]:
%%writefile ch20/method_is_descriptor.py
import collections


class Text(collections.UserString):
  def __repr__(self):
    return 'Text({!r})'.format(self.data)

  def reverse(self):
    return self[::-1]

Writing ch20/method_is_descriptor.py


###### 示例 20-15测试一个方法

In [ ]:
from ch20.method_is_descriptor import Text
word = Text('forward')
word  # Text 实例的 repr 方法返回一个类似 Text 构造方法调用的字符串,可用于创建相同的实例

Text('forward')

In [ ]:
word.reverse()  # reverse 方法返回反向拼写的单词

Text('drawrof')

In [ ]:
Text.reverse(Text('backward'))  # 在类上调用方法相当于调用函数

Text('drawkcab')

In [ ]:
type(Text.reverse), type(word.reverse)  # 类型不同,一个是 function,一个是 method

(function, method)

In [ ]:
[Text.reverse(item) for item in ['repaid', (10, 20, 30), Text('stressed')]]  # Text.reverse 相当于函数,甚至可以处理 Text 实例之外的其他对象

['diaper', (30, 20, 10), Text('desserts')]

In [ ]:
Text.reverse.__get__(word)  # 函数都是非覆盖型描述符。在函数上调用 __get__ 方法时传入实例,得到的是绑定到那个实例上的方法

<bound method Text.reverse of Text('forward')>

In [ ]:
Text.reverse.__get__(None, Text)  # 调用函数的 __get__ 方法时,如果 instance 参数的值是 None,那么得到的是函数本身

<function ch20.method_is_descriptor.Text.reverse>

In [ ]:
word.reverse  # word.reverse 表达式其实会调用 Text.reverse.__get__(word),返回对应的绑定方法

<bound method Text.reverse of Text('forward')>

In [ ]:
word.reverse.__self__  # 绑定方法对象有个 __self__ 属性,其值是调用这个方法的实例引用

Text('forward')

In [ ]:
word.reverse.__func__ is Text.reverse  # 绑定方法的 __func__ 属性是依附在托管类上那个原始函数的引用

True

- 绑定方法对象还有个 `__call__` 方法, 用于处理真正的调用过程
  - 这个方法会调用 `__func__` 属性引用的原始函数
  - 把函数的第一个参数设为绑定方法的 `__self__` 属性
    - 这就是形参 `self` 的隐式绑定方式


## 20.4 描述符用法建议

###### 使用特性以保持简单

- 内置的 `property` 类创建的其实是覆盖型描述符,`__set__` 方法和`__get__` 方法都实现了
- 不定义设值方法时, 特性的`__set__` 方法默认抛出 `AttributeError: can't set attribute`
  - 因此创建只读属性最简单的方式是使用特性


###### 只读描述符必须有 `__set__` 方法

- 如果使用描述符类实现只读属性, `__get__` 和 `__set__`两个方法必须都定义
- 否则,实例的同名属性会遮盖描述符
- 只读属性的 `__set__` 方法只需抛出 `AttributeError` 异常,并提供合适的错误消息


###### 用于验证的描述符可以只有 `__set__` 方法

- 对仅用于验证的描述符来说,`__set__` 方法应该检查 `value` 参数获得的值,如果有效,使用描述符实例的名称为键,直接在实例的`__dict__` 属性中设置
- 这样, 从实例中读取同名属性的速度很快, 因为不用经过 `__get__` 方法处理

###### 仅有 `__get__` 方法的描述符可以实现高效缓存

- 如果只编写了 `__get__` 方法, 那么创建的是非覆盖型描述符
- 这种描述符可用于执行某些耗费资源的计算, 然后为实例设置同名属性, 缓存结果
- 同名实例属性会遮盖描述符, 因此后续访问会直接从实例的 `__dict__` 属性中获取值,而不会再触发描述符的 `__get__` 方法


###### 非特殊的方法可以被实例属性遮盖

- 由于函数和方法只实现了 `__get__` 方法, 它们不会处理同名实例属性的赋值操作
  - 因此,像 `my_obj.the_method = 7` 这样简单赋值之后, 后续通过该实例访问 `the_method` 得到的是数字 `7`
  - 但是不影响类或其他实例
- 然而, 特殊方法不受这个问题的影响
  - 解释器只会在类中寻找特殊的方法
    - `repr(x)` 执行的其实是`x.__class__.__repr__(x)`
      - 因此 `x` 的 `__repr__` 属性对 `repr(x)` 方法调用没有影响
    - 出于同样的原因,实例的 `__getattr__` 属性不会破坏常规的属性访问规则
- 如果要创建大量动态属性, 属性名称从不受自己控制的数据中获取(像本章前面那样), 则应该知道这种行为
  - 但只要通过类访问, 类方法就是安全的

## 20.5 描述符的文档字符串和覆盖删除操作

- 定制各个描述符实例的帮助文档特别难

In [ ]:
help(LineItem.weight)

Help on Quantity in module ch20.model_v5 object:

class Quantity(Validated)
 |  a number greater than zero
 |  
 |  Method resolution order:
 |      Quantity
 |      Validated
 |      abc.ABC
 |      AutoStorage
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  validate(self, instance, value)
 |      return validated value or raise ValueError
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Validated:
 |  
 |  __set__(self, instance, value)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from abc.ABC:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  -----------------------------

In [ ]:
help(LineItem)

Help on class LineItem in module ch20.bulkfood_v5:

class LineItem(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, description, weight, price)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  subtotal(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  description
 |      a string with at least one non-space character
 |  
 |  price
 |      a number greater than zero
 |  
 |  weight
 |      a number greater than zero

